# K2-18 

In [ ]:
import lightkurve as lk
from glob import glob
import ombre as om
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk

import astropy.units as u

In [ ]:
name = 'K2-18'

In [ ]:
fnames = np.asarray(glob(f'/Users/ch/HST/data/{name}/*.fits'))
fnames = np.sort(fnames)
obs = om.Observation.from_files(fnames)

In [ ]:
obs

In [ ]:
clc = lk.search_lightcurve(name, mission='K2', author='EVEREST').download()
clc.flux = clc.flux.value * u.electron/u.s
clc.flux_err = clc.flux.value * u.electron/u.s
clc = clc.flatten(51)
clc = clc[clc.flux.value > 0.97]
clc = clc.remove_outliers(sigma_upper=3, sigma_lower=1e2)

In [ ]:
k = np.abs(clc.fold(obs.period, obs.t0 - 2454833).time) < 0.5
k = k[np.argsort(clc.fold(obs.period, obs.t0 - 2454833).time_original)]

In [ ]:
clc.fold(obs.period, obs.t0 - 2454833).scatter()

In [ ]:
obs.fit_transit(x_suppl=clc.time.value[k] + 2454833,
                y_suppl=clc.flux.value[k],
                yerr_suppl=clc.flux_err.value[k],
                exptime_suppl=np.median(np.diff(clc.time.value)))

In [ ]:
obs.plot()

In [ ]:
obs.fit_model()

In [ ]:
obs.plot_spectra()

In [ ]:
om.Spectra([visit.transmission_spectrum for visit in obs]).flatten().bin(bins=np.linspace(0.8, 1.7, 300)).plot()

In [ ]:
spec = obs.stellar_spectra
spec.plot()
spec.hdulist.writeto(f'results/{name}_stellar_spectrum.fits', overwrite=True)

In [ ]:
from astropy.io import fits
try:
    hdulist = om.Spectra([visit.transmission_spectrum for visit in obs if visit.transmission_spectrum.spec.sum() != 0]).hdulist
    hdulist.writeto(f'results/{name}_transmission.fits', overwrite=True)
except:
    pass
try:    
    hdulist = om.Spectra([visit.emission_spectrum for visit in obs if visit.emission_spectrum.spec.sum() != 0]).hdulist
    hdulist.writeto(f'results/{name}_emission.fits', overwrite=True)
except:
    pass

In [ ]:
for visit in obs:
    visit.diagnose();
    plt.show();